In [1]:
%cd ..

/home/atanu/Documents/swatah/fire-and-smoke-detection


In [2]:
import cv2
import time
import onnxruntime as ort

from utils.process import preprocessing, postprocessing

ONNX_MODEL_PATH = "models/smoke_tiny_lrelu.onnx"
CLASS_NAMES = ["BG", "Fire", "Smoke"]
COLORS = ["red", "yellow", "white"]
CV2COLORS = [(255, 255, 255), (0, 255, 255), (255, 255, 255)]
IMAGE_SHAPE = (640, 640)
video_path = "data/videos/sample_01.mp4"

In [3]:
session = ort.InferenceSession(ONNX_MODEL_PATH)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [4]:
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()


prev_time = 0

while True:
    ret, image = cap.read()

    if not ret:
        print("End of video or cannot fetch the frame.")
        break

    current_time = time.time()
    fps = 1 / (current_time - prev_time) if prev_time else 0
    prev_time = current_time

    # preprocessing
    tensor_input, image, ratios = preprocessing(image, IMAGE_SHAPE)

    # predictions
    predictions = session.run([output_name], {input_name: tensor_input})

    # postprocessing
    outputs = postprocessing(predictions[0], ratios)

    for detections in outputs:

        if detections.ndim == 1:
            detections = detections.reshape(1, -1)

            for det in detections:
                x1, y1, x2, y2 = map(int, det[:4])
                class_id = int(det[8])
                # score = det[4] * det[5 + class_id]
                score = det[4]
                label = f"{CLASS_NAMES[class_id]}: {score:.2f}"

                cv2.rectangle(image, (x1, y1), (x2, y2), CV2COLORS[class_id], 1)

                cv2.putText(
                    image,
                    label,
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    CV2COLORS[class_id],
                    1,
                )

    cv2.putText(
        image, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2
    )
    cv2.imshow("Video Playback", image)
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()